In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

#### ukb46987_new에서 Q15, H40 (glaucoma) 진단자, 진단일 열 생성

H40.0 ~ H40.9            
Q15.0, Q15.8, Q15.9  -> Q15 포함 x

In [2]:
# 8분 이상 걸림

ukb46987 = pd.read_sas('SAS\\ukb46987_new.sas7bdat', format = 'sas7bdat', encoding = 'cp1252')
ukb46987

,eid,41270-0.0,41270-0.1,41270-0.2,41270-0.3,41270-0.4,41270-0.5,41270-0.6,41270-0.7,41270-0.8,...,41280-0.213,41280-0.214,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222
0,1000015,D693,E119,E780,I10,I200,I209,I251,I259,I350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,D128,E780,J459,K621,N40,S609,W290,Z800,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,D127,D508,D509,D649,E669,F412,F431,G933,I10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,I839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,E039,Z038,Z538,Z873,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,I083,I10,I278,I48,S9230,W199,Z867,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,J189,K573,N393,T814,Y838,Z121,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = ukb46987.copy()

In [4]:
df2 = df.iloc[:, :224]
df2

,eid,41270-0.0,41270-0.1,41270-0.2,41270-0.3,41270-0.4,41270-0.5,41270-0.6,41270-0.7,41270-0.8,...,41270-0.213,41270-0.214,41270-0.215,41270-0.216,41270-0.217,41270-0.218,41270-0.219,41270-0.220,41270-0.221,41270-0.222
0,1000015,D693,E119,E780,I10,I200,I209,I251,I259,I350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,D128,E780,J459,K621,N40,S609,W290,Z800,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,D127,D508,D509,D649,E669,F412,F431,G933,I10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,I839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,E039,Z038,Z538,Z873,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,I083,I10,I278,I48,S9230,W199,Z867,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,J189,K573,N393,T814,Y838,Z121,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df4 = df.iloc[:, 224:]
df4

,41280-0.0,41280-0.1,41280-0.2,41280-0.3,41280-0.4,41280-0.5,41280-0.6,41280-0.7,41280-0.8,41280-0.9,...,41280-0.213,41280-0.214,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222
0,1999-07-22,2018-06-07,2002-10-30,1999-09-23,2002-10-30,2002-10-17,2002-10-30,2018-06-07,2018-06-07,2014-12-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-16,2014-10-24,2014-10-24,2015-01-16,2014-10-24,2004-05-27,2004-05-27,2015-01-16,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-13,2018-10-16,2018-10-30,2018-11-20,2019-02-05,2020-02-13,2018-10-09,2018-10-09,2018-11-20,2018-10-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,1999-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,2014-09-30,2014-09-30,2014-09-30,2014-09-30,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,2013-12-30,2013-12-30,2013-12-30,2013-12-30,2015-09-30,2015-09-30,2013-12-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,1999-01-25,2015-10-17,2002-03-24,2002-04-06,2002-04-06,2015-10-17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# colnames = df2.columns.tolist()
# colnames.remove('eid')

# n18 = 0
# for row in range(len(df2)): 
#     for col in colnames: 
#         print(df2[col][row])
#         if df2[col][row] is not np.nan: 
#             disease = df2[col][row].decode('euc-kr')
#             if disease.startswith('N18') == True: 
#                 n18 += 1
#                 break

In [17]:
TT = pd.DataFrame(df2['41270-0.0'].loc[(df2['41270-0.0'].str.startswith('H400')) & (df2['41270-0.0'].isna() == False)])

In [18]:
# df2['H400'] = 0
df2['H401'] = 0
df2['H402'] = 0
df2['H403'] = 0
df2['H404'] = 0
df2['H405'] = 0
df2['H406'] = 0
df2['H407'] = 0
df2['H408'] = 0
df2['H409'] = 0

In [19]:
df2

,eid,41270-0.0,41270-0.1,41270-0.2,41270-0.3,41270-0.4,41270-0.5,41270-0.6,41270-0.7,41270-0.8,...,41270-0.222,H401,H402,H403,H404,H405,H406,H407,H408,H409
0,1000015,D693,E119,E780,I10,I200,I209,I251,I259,I350,...,NaN,0,0,0,0,0,0,0,0,0
1,1000027,D128,E780,J459,K621,N40,S609,W290,Z800,NaN,...,NaN,0,0,0,0,0,0,0,0,0
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
4,1000053,D127,D508,D509,D649,E669,F412,F431,G933,I10,...,NaN,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,I839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
502472,6025340,E039,Z038,Z538,Z873,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
502473,6025353,I083,I10,I278,I48,S9230,W199,Z867,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
502474,6025364,J189,K573,N393,T814,Y838,Z121,NaN,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0


In [20]:
colnames = df2.columns.tolist()
colnames.remove('eid')

In [55]:
# for col in colnames: 
#     df2.loc[df2[col] == 'H400', 'H400'] = 1  # 전체 222개 칼럼 순회하면서 H400 있으면 1 넣어주기
#     df2.loc[df2[col] == 'H400', 'H400_index'] = col[6:]  # 41280 변수에서 H40 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

#     #df2["N180"] = df2.apply(lambda x: 1 if x[col] == 'N180' else 0, axis=0)

In [56]:
# df2['H400'].value_counts() 

0    499527
1      2949
Name: H400, dtype: int64

In [57]:
# # 인덱스랑 진단여부 갯수 똑같은 거 확인

# df2['H400_index'].isna().sum()

499527

In [21]:
for col in colnames: 
    df2.loc[df2[col] == 'H401', 'H401'] = 1  # 전체 222개 칼럼 순회하면서 H401 있으면 1 넣어주기
    df2.loc[df2[col] == 'H401', 'H401_index'] = col[6:]  # 41280 변수에서 H40 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [22]:
for col in colnames: 
    df2.loc[df2[col] == 'H402', 'H402'] = 1  # 전체 222개 칼럼 순회하면서 H402 있으면 1 넣어주기
    df2.loc[df2[col] == 'H402', 'H402_index'] = col[6:]  # 41280 변수에서 H40 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [23]:
for col in colnames: 
    df2.loc[df2[col] == 'H403', 'H403'] = 1  # 전체 222개 칼럼 순회하면서 H403 있으면 1 넣어주기
    df2.loc[df2[col] == 'H403', 'H403_index'] = col[6:]  # 41280 변수에서 H40 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [25]:
for col in colnames: 
    df2.loc[df2[col] == 'H404', 'H404'] = 1  # 전체 222개 칼럼 순회하면서 H404 있으면 1 넣어주기
    df2.loc[df2[col] == 'H404', 'H404_index'] = col[6:]  # 41280 변수에서 H40 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [26]:
for col in colnames: 
    df2.loc[df2[col] == 'H405', 'H405'] = 1  # 전체 222개 칼럼 순회하면서 H405 있으면 1 넣어주기
    df2.loc[df2[col] == 'H405', 'H405_index'] = col[6:]  # 41280 변수에서 H40 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [27]:
for col in colnames: 
    df2.loc[df2[col] == 'H406', 'H406'] = 1  # 전체 222개 칼럼 순회하면서 H406 있으면 1 넣어주기
    df2.loc[df2[col] == 'H406', 'H406_index'] = col[6:]  # 41280 변수에서 H40 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [28]:
for col in colnames: 
    df2.loc[df2[col] == 'H407', 'H407'] = 1  # 전체 222개 칼럼 순회하면서 H407 있으면 1 넣어주기
    df2.loc[df2[col] == 'H407', 'H407_index'] = col[6:]  # 41280 변수에서 H40 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [29]:
for col in colnames: 
    df2.loc[df2[col] == 'H408', 'H408'] = 1  # 전체 222개 칼럼 순회하면서 H408 있으면 1 넣어주기
    df2.loc[df2[col] == 'H408', 'H408_index'] = col[6:]  # 41280 변수에서 H40 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [30]:
for col in colnames: 
    df2.loc[df2[col] == 'H409', 'H409'] = 1  # 전체 222개 칼럼 순회하면서 H409 있으면 1 넣어주기
    df2.loc[df2[col] == 'H409', 'H409_index'] = col[6:]  # 41280 변수에서 H40 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [31]:
# for col in colnames: 
#     df2.loc[df2[col] == 'Q150', 'Q150'] = 1  # 전체 222개 칼럼 순회하면서 Q150 있으면 1 넣어주기
#     df2.loc[df2[col] == 'Q150', 'Q150_index'] = col[6:]  # 41280 변수에서 Q15 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기
# for col in colnames: 
#     df2.loc[df2[col] == 'Q158', 'Q158'] = 1  # 전체 222개 칼럼 순회하면서 Q158 있으면 1 넣어주기
#     df2.loc[df2[col] == 'Q158', 'Q158_index'] = col[6:]  # 41280 변수에서 Q15 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기
# for col in colnames: 
#     df2.loc[df2[col] == 'Q159', 'Q159'] = 1  # 전체 222개 칼럼 순회하면서 Q159 있으면 1 넣어주기
#     df2.loc[df2[col] == 'Q159', 'Q159_index'] = col[6:]  # 41280 변수에서 Q15 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [32]:
# colnames = df2.columns.tolist()
# colnames.remove('eid')

# for col in colnames: 
#     print(col, ' ', df2.loc[df2[col] == 'N181', col].count())

In [33]:
# for col in colnames: 
#     df["N180"] = df.apply(lambda x: 1 if x[col] == 'N180' else 0, axis = 1)
#     df["N181"] = df.apply(lambda x: 1 if x[col] == 'N181' else 0, axis = 1)
#     df["N182"] = df.apply(lambda x: 1 if x[col] == 'N182' else 0, axis = 1)
#     df["N183"] = df.apply(lambda x: 1 if x[col] == 'N183' else 0, axis = 1)
#     df["N184"] = df.apply(lambda x: 1 if x[col] == 'N184' else 0, axis = 1)
#     df["N185"] = df.apply(lambda x: 1 if x[col] == 'N185' else 0, axis = 1)
#     df["N186"] = df.apply(lambda x: 1 if x[col] == 'N186' else 0, axis = 1)
#     df["N187"] = df.apply(lambda x: 1 if x[col] == 'N187' else 0, axis = 1)
#     df["N188"] = df.apply(lambda x: 1 if x[col] == 'N188' else 0, axis = 1)
#     df["N189"] = df.apply(lambda x: 1 if x[col] == 'N189' else 0, axis = 1)

In [34]:
# df2["H400"].value_counts()

In [35]:
# df2['H400_index'].isna().sum()

In [36]:
df2["H401"].value_counts()

0    499207
1      3269
Name: H401, dtype: int64

In [37]:
df2['H401_index'].isna().sum()

499207

In [38]:
df2["H402"].value_counts()

0    500830
1      1646
Name: H402, dtype: int64

In [39]:
df2['H402_index'].isna().sum()

500830

In [40]:
df2["H403"].value_counts()

0    502434
1        42
Name: H403, dtype: int64

In [41]:
df2['H403_index'].isna().sum()

502434

In [42]:
df2["H404"].value_counts()

0    502413
1        63
Name: H404, dtype: int64

In [43]:
df2['H404_index'].isna().sum()

502413

In [44]:
df2["H405"].value_counts()

0    502178
1       298
Name: H405, dtype: int64

In [45]:
df2['H405_index'].isna().sum()

502178

In [46]:
df2["H406"].value_counts()

0    502457
1        19
Name: H406, dtype: int64

In [47]:
df2['H406_index'].isna().sum()

502457

In [48]:
df2["H407"].value_counts()

0    502476
Name: H407, dtype: int64

In [49]:
df2['H407_index'].isna().sum()

502476

In [50]:
df2["H408"].value_counts()

0    502295
1       181
Name: H408, dtype: int64

In [51]:
df2['H408_index'].isna().sum()

502295

In [52]:
df2["H409"].value_counts()

0    494768
1      7708
Name: H409, dtype: int64

In [53]:
df2['H409_index'].isna().sum()

494768

In [54]:
# df2["Q150"].value_counts()
# df2['Q150_index'].isna().sum()
# df2["Q158"].value_counts()
# df2['Q158_index'].isna().sum()
# df2["Q159"].value_counts()
# df2['Q159_index'].isna().sum()

In [55]:
# df_final = df2[['eid', 'H400', 'H401', 'H402', 'H403', 'H404', 'H405', 'H406', 'H407', 'H408', 'H409',  
#                 'H400_index', 'H401_index', 'H402_index', 'H403_index', 'H404_index', 'H405_index', 'H406_index', 'H407_index', 'H408_index', 'H409_index', 
#                 ]]
# df_final

df_final = df2[['eid', 'H401', 'H402', 'H403', 'H404', 'H405', 'H406', 'H407', 'H408', 'H409',  
                'H401_index', 'H402_index', 'H403_index', 'H404_index', 'H405_index', 'H406_index', 'H407_index', 'H408_index', 'H409_index', 
                ]]
df_final

,eid,H401,H402,H403,H404,H405,H406,H407,H408,H409,H401_index,H402_index,H403_index,H404_index,H405_index,H406_index,H407_index,H408_index,H409_index
0,1000015,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df4

,41280-0.0,41280-0.1,41280-0.2,41280-0.3,41280-0.4,41280-0.5,41280-0.6,41280-0.7,41280-0.8,41280-0.9,...,41280-0.213,41280-0.214,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222
0,1999-07-22,2018-06-07,2002-10-30,1999-09-23,2002-10-30,2002-10-17,2002-10-30,2018-06-07,2018-06-07,2014-12-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-16,2014-10-24,2014-10-24,2015-01-16,2014-10-24,2004-05-27,2004-05-27,2015-01-16,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-13,2018-10-16,2018-10-30,2018-11-20,2019-02-05,2020-02-13,2018-10-09,2018-10-09,2018-11-20,2018-10-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,1999-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,2014-09-30,2014-09-30,2014-09-30,2014-09-30,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,2013-12-30,2013-12-30,2013-12-30,2013-12-30,2015-09-30,2015-09-30,2013-12-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,1999-01-25,2015-10-17,2002-03-24,2002-04-06,2002-04-06,2015-10-17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df5 = pd.concat([df_final, df4], axis = 1)
df5

,eid,H401,H402,H403,H404,H405,H406,H407,H408,H409,...,41280-0.213,41280-0.214,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222
0,1000015,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# dftmp = df5[['H400_index', 'H401_index', 'H402_index', 'H403_index', 'H404_index', 'H405_index', 'H406_index', 'H407_index', 'H408_index', 'H409_index']].copy()
dftmp = df5[['H401_index', 'H402_index', 'H403_index', 'H404_index', 'H405_index', 'H406_index', 'H407_index', 'H408_index', 'H409_index']].copy()

In [92]:
# dftmp['H400_index'].unique()

array([nan, '0.15', '0.2', '0.6', '0.3', '0.0', '0.13', '0.7', '0.1',
       '0.5', '0.4', '0.11', '0.10', '0.8', '0.9', '0.18', '0.12', '0.14',
       '0.19', '0.21', '0.17', '0.16', '0.20', '0.22', '0.23', '0.24'],
      dtype=object)

In [ ]:
#### TEST

coltest = ['idx1', 'idx2', 'idx3', '41280-1.0', '41280-5.0', '41280-9.0']
value = [[1, np.nan, np.nan, '2010-01-01', np.nan, np.nan], [np.nan, 5, np.nan, np.nan, '2010-03-03', np.nan], [np.nan, np.nan, 9, np.nan, np.nan, '2021-01-01'], [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]]
testdf = pd.DataFrame(columns = coltest, data=value)

# vltest = testdf[col].sum(axis = 1).tolist()
# vltestdf = pd.DataFrame(vltest)
# vltestdf[0].unique()

testdf['N18_idx'] = testdf.stack().groupby(level=0).first().reindex(testdf.index)   # 각 행에서 Nan 값이 아닌 유일한 값 가져오기
testdf
# testdf['N18_Date'] = list(map(lambda y: np.nan if y is np.nan else testdf['41280-' + str(y)].values[0], testdf['N18_idx']))

In [ ]:
testdf['eid'] = ['A', 'B', 'C', 'D']
valuelist = testdf['N18_idx'].tolist()

testdf['N18_Date'] = list(map(lambda x, y: np.nan if y is np.nan else testdf.loc[testdf['eid'] == x, '41280-' + str(y)].values[0], testdf['eid'], valuelist))

In [ ]:
testdf

In [ ]:
########## test end

In [61]:
# column_list = ['H400_index', 'H401_index', 'H402_index', 'H403_index', 'H404_index', 'H405_index', 'H406_index', 'H407_index', 'H408_index', 'H409_index']

# dftmp['H40_idx'] = dftmp.stack().groupby(level=0).first().reindex(dftmp.index)  # 각 행에서 Nan 값이 아닌 유일한 값 가져오기

column_list = ['H401_index', 'H402_index', 'H403_index', 'H404_index', 'H405_index', 'H406_index', 'H407_index', 'H408_index', 'H409_index']

dftmp['H40_idx'] = dftmp.stack().groupby(level=0).first().reindex(dftmp.index)  # 각 행에서 Nan 값이 아닌 유일한 값 가져오기

In [62]:
dftmp['H40_idx'].unique()

array([nan, '0.4', '0.2', '0.9', '0.14', '0.7', '0.1', '0.0', '0.8',
       '0.11', '0.6', '0.3', '0.13', '0.24', '0.23', '0.5', '0.10',
       '0.16', '0.21', '0.12', '0.15', '0.20', '0.19', '0.17', '0.18',
       '0.22'], dtype=object)

In [63]:
dftmp

,H401_index,H402_index,H403_index,H404_index,H405_index,H406_index,H407_index,H408_index,H409_index,H40_idx
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
502471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df5 = pd.concat([df5, dftmp['H40_idx']], axis = 1)
df5

,eid,H401,H402,H403,H404,H405,H406,H407,H408,H409,...,41280-0.214,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222,H40_idx
0,1000015,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
c = df5.columns.tolist()
for col in c: 
    print(col)

eid
H401
H402
H403
H404
H405
H406
H407
H408
H409
H401_index
H402_index
H403_index
H404_index
H405_index
H406_index
H407_index
H408_index
H409_index
41280-0.0
41280-0.1
41280-0.2
41280-0.3
41280-0.4
41280-0.5
41280-0.6
41280-0.7
41280-0.8
41280-0.9
41280-0.10
41280-0.11
41280-0.12
41280-0.13
41280-0.14
41280-0.15
41280-0.16
41280-0.17
41280-0.18
41280-0.19
41280-0.20
41280-0.21
41280-0.22
41280-0.23
41280-0.24
41280-0.25
41280-0.26
41280-0.27
41280-0.28
41280-0.29
41280-0.30
41280-0.31
41280-0.32
41280-0.33
41280-0.34
41280-0.35
41280-0.36
41280-0.37
41280-0.38
41280-0.39
41280-0.40
41280-0.41
41280-0.42
41280-0.43
41280-0.44
41280-0.45
41280-0.46
41280-0.47
41280-0.48
41280-0.49
41280-0.50
41280-0.51
41280-0.52
41280-0.53
41280-0.54
41280-0.55
41280-0.56
41280-0.57
41280-0.58
41280-0.59
41280-0.60
41280-0.61
41280-0.62
41280-0.63
41280-0.64
41280-0.65
41280-0.66
41280-0.67
41280-0.68
41280-0.69
41280-0.70
41280-0.71
41280-0.72
41280-0.73
41280-0.74
41280-0.75
41280-0.76
41280-0.77
4128

In [66]:
# 열별로 진단받은 날짜 추가하기

valuelist = df5['H40_idx'].tolist()
df5['H40_Date'] = list(map(lambda x, y: np.nan if y is np.nan else df5.loc[df5['eid'] == x, '41280-' + str(y)].values[0], df5['eid'], valuelist))

In [67]:
len(df5['H40_Date'].unique())

4199

In [68]:
df5

,eid,H401,H402,H403,H404,H405,H406,H407,H408,H409,...,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222,H40_idx,H40_Date
0,1000015,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
# df6 = df5[['eid', 
# 'H400', 'H401', 'H402', 'H403', 'H404', 'H405', 'H406', 'H407', 'H408', 'H409', 
# 'H400_index', 'H401_index', 'H402_index', 'H403_index', 'H404_index', 'H405_index', 'H406_index', 'H407_index', 'H408_index', 'H409_index',
# 'H40_idx', 'H40_Date']]

df6 = df5[['eid', 
'H401', 'H402', 'H403', 'H404', 'H405', 'H406', 'H407', 'H408', 'H409', 
'H401_index', 'H402_index', 'H403_index', 'H404_index', 'H405_index', 'H406_index', 'H407_index', 'H408_index', 'H409_index',
'H40_idx', 'H40_Date']]

In [70]:
df6

,eid,H401,H402,H403,H404,H405,H406,H407,H408,H409,...,H402_index,H403_index,H404_index,H405_index,H406_index,H407_index,H408_index,H409_index,H40_idx,H40_Date
0,1000015,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
df6.to_csv('0. data\\ukb46987_H40preprocessed.csv', encoding = 'euc-kr', index = False)

#### 2021.12.06 ~

In [72]:
df = pd.read_csv('0. data\\ukb46987_H40preprocessed.csv', encoding = 'euc-kr')
df

,eid,H401,H402,H403,H404,H405,H406,H407,H408,H409,...,H402_index,H403_index,H404_index,H405_index,H406_index,H407_index,H408_index,H409_index,H40_idx,H40_Date
0,1000015,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
# def conditions(df):
#     if (df['H400'] == 1) or (df['H401'] == 1) or (df['H402'] == 1) or (df['H403'] == 1) or (df['H404'] == 1) or (df['H405'] == 1) or (df['H406'] == 1) or (df['H407'] == 1) or (df['H408'] == 1) or (df['H409'] == 1):
#         return 1
#     else:
#         return 0

def conditions(df):
    if (df['H401'] == 1) or (df['H402'] == 1) or (df['H403'] == 1) or (df['H404'] == 1) or (df['H405'] == 1) or (df['H406'] == 1) or (df['H407'] == 1) or (df['H408'] == 1) or (df['H409'] == 1):
        return 1
    else:
        return 0

In [74]:
df['H40'] = df.apply(conditions, axis=1)

In [75]:
df['H40'].value_counts()

0    491918
1     10558
Name: H40, dtype: int64

In [76]:
df['H40_Date'].fillna(0, inplace = True)

In [77]:
df.loc[df['H40_Date'] == 0, 'H40_Date'].count()

491918

In [78]:
df

,eid,H401,H402,H403,H404,H405,H406,H407,H408,H409,...,H403_index,H404_index,H405_index,H406_index,H407_index,H408_index,H409_index,H40_idx,H40_Date,H40
0,1000015,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,1000027,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,1000039,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,1000040,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,1000053,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
502472,6025340,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
502473,6025353,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
502474,6025364,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [79]:
df_final = df[['eid', 'H40_Date', 'H40']]
df_final

,eid,H40_Date,H40
0,1000015,0,0
1,1000027,0,0
2,1000039,0,0
3,1000040,0,0
4,1000053,0,0
...,...,...,...
502471,6025339,0,0
502472,6025340,0,0
502473,6025353,0,0
502474,6025364,0,0


In [80]:
df_final['H40'].value_counts()

0    491918
1     10558
Name: H40, dtype: int64

In [82]:
df_final.to_csv('0. data\\ukb46987_H40preprocessed.csv', encoding = 'euc-kr', index = False)

#### HTN drug preprocessing

In [3]:
ukb45783_drug = pd.read_sas('SAS\\ukb45783_drug.sas7bdat', format = 'sas7bdat')
ukb45783_drug

KeyboardInterrupt: 

Exception ignored in: 'pandas.io.sas._sas.Parser.read_next_page'
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\pandas\io\sas\sas7bdat.py", line 729, in _read_next_page
    def _read_next_page(self):
KeyboardInterrupt: 


In [44]:
df = ukb45783_drug.copy()

In [45]:
# HTN drug code - 공단 약제 list 기준

HTN_drugcode_preprocessed_FINAL = pd.read_csv('data\\HTN_drugcode_preprocessed_FINAL.csv', encoding = 'euc-kr')
HTN_drugcode_preprocessed_FINAL

,주성분코드,주성분,Category,UK Biobank Count,UK Biobank code
0,104201ATB,1042.0,alacepril,NaN,NaN
1,107601ATB,1076.0,amlodipine besylate,NaN,NaN
2,107901ATB,1079.0,amosulalol hydrochloride,NaN,NaN
3,111402ATB,1114.0,atenolol,20583.0,1.140867e+09
4,114001ACH,1140.0,barnidipine hydrochloride,NaN,NaN
...,...,...,...,...,...
68,180230BIJ,1802.0,labetalol hydrochloride,NaN,NaN
69,229330BIJ,2293.0,sodium nitroprusside,NaN,NaN
70,512430BIJ,5124.0,treprostinil,NaN,NaN
71,471430CSI,4714.0,iloprost,NaN,NaN


In [46]:
# 1213 HTN drug 기준 확대
# HTN drug code - 약학정보원 고혈압 약제 list 기준 재추출

HTN_drugcode_preprocessed_FINAL = pd.read_excel('data\\고혈압_약제성분_약학정보원기준.xlsx')
HTN_drugcode_preprocessed_FINAL

,http://www.health.kr/searchIngredient/KPICeffect.asp,Unnamed: 1,Unnamed: 2,UK Biobank count,UK Biobank code
0,베타-차단제 (β-Adrenergic Receptor Blockers),β1-비선택성,Carteolol,35.0,1.140880e+09
1,NaN,NaN,Nadolol,16.0,1.140860e+09
2,NaN,NaN,Propranolol,3750.0,1.140880e+09
3,NaN,β1-선택성,Acebutolol,19.0,1.140867e+09
4,NaN,NaN,Atenolol,20583.0,1.140867e+09
5,NaN,NaN,Betaxolol,33.0,1.140880e+09
6,NaN,NaN,Bisoprolol,9638.0,1.140880e+09
7,NaN,NaN,Celiprolol,145.0,1.140880e+09
8,NaN,NaN,Metoprolol,1026.0,1.140880e+09
9,NaN,NaN,Nebivolol,468.0,1.141164e+09


In [47]:
htn_drugcode = HTN_drugcode_preprocessed_FINAL['UK Biobank code'].dropna()
htn_drugcode = htn_drugcode.tolist()
htn_drugcode

[1140879822.0,
 1140860192.0,
 1140879842.0,
 1140866724.0,
 1140866738.0,
 1140879758.0,
 1140879760.0,
 1140879762.0,
 1140879818.0,
 1141164276.0,
 1140909368.0,
 1140879824.0,
 1140879794.0,
 1140879798.0,
 1140860750.0,
 1140860696.0,
 1140860882.0,
 1140888552.0,
 1140888560.0,
 1140860806.0,
 1140860904.0,
 1141171336.0,
 1141152998.0,
 1140916356.0,
 1141193282.0,
 1141166006.0,
 1141145660.0,
 1140879806.0,
 1140888510.0,
 1140861088.0,
 1140879810.0,
 1140879802.0,
 1140861276.0,
 1141153026.0,
 1140888686.0,
 1140860532.0,
 1140860470.0]

In [48]:
colnames = df.columns.tolist()
colnames.remove('eid')

In [49]:
for col in colnames: 
    for val in htn_drugcode: 
        df.loc[df[col] == val, str(val)] = 1  # 전체 칼럼 순회하면서 drug code 있으면 칼럼 만들어서 1 넣어주기

In [12]:
df

,eid,20003-0.0,20003-0.1,20003-0.2,20003-0.3,20003-0.4,20003-0.5,20003-0.6,20003-0.7,20003-0.8,...,1140879806.0,1140888510.0,1140861088.0,1140879810.0,1140879802.0,1140861276.0,1141153026.0,1140888686.0,1140860532.0,1140860470.0
0,6025371.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000015.0,1.141165e+09,1.140922e+09,1.140868e+09,1.140910e+09,1.141195e+09,1.140865e+09,1.140916e+09,1.140884e+09,2.038460e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000027.0,1.141146e+09,1.141168e+09,1.140862e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000039.0,1.140868e+09,1.141195e+09,1.140871e+09,1.140861e+09,1.140879e+09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000040.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502481,6025327.0,1.140853e+09,1.193000e+03,1.140885e+09,1.140910e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502482,6025339.0,1.141153e+09,1.140910e+09,1.140910e+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502483,6025340.0,1.140910e+09,1.141178e+09,1.140917e+09,1.140853e+09,1.201000e+03,1.140871e+09,1.140910e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502484,6025353.0,1.140866e+09,2.038460e+09,1.140861e+09,1.140879e+09,1.140862e+09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# 엑셀로 정리해둔 명수랑 얼추 맞음

for val in htn_drugcode: 
    print(df[str(val)].value_counts())

1.0    35
Name: 1140879822.0, dtype: int64
1.0    15
Name: 1140860192.0, dtype: int64
1.0    3547
Name: 1140879842.0, dtype: int64
1.0    18
Name: 1140866724.0, dtype: int64
1.0    19526
Name: 1140866738.0, dtype: int64
1.0    33
Name: 1140879758.0, dtype: int64
1.0    8976
Name: 1140879760.0, dtype: int64
1.0    131
Name: 1140879762.0, dtype: int64
1.0    990
Name: 1140879818.0, dtype: int64
1.0    437
Name: 1141164276.0, dtype: int64
1.0    390
Name: 1140909368.0, dtype: int64
1.0    71
Name: 1140879824.0, dtype: int64
1.0    68
Name: 1140879794.0, dtype: int64
1.0    176
Name: 1140879798.0, dtype: int64
1.0    225
Name: 1140860750.0, dtype: int64
1.0    14599
Name: 1140860696.0, dtype: int64
1.0    24
Name: 1140860882.0, dtype: int64
1.0    3854
Name: 1140888552.0, dtype: int64
1.0    7375
Name: 1140888560.0, dtype: int64
1.0    25428
Name: 1140860806.0, dtype: int64
1.0    293
Name: 1140860904.0, dtype: int64
1.0    290
Name: 1141171336.0, dtype: int64
1.0    3896
Name: 1141152998.

In [51]:
htn_drugcode_str = list(map(str, htn_drugcode))
htn_drugcode_str.insert(0, 'eid')
htn_drugcode_str

['eid',
 '1140879822.0',
 '1140860192.0',
 '1140879842.0',
 '1140866724.0',
 '1140866738.0',
 '1140879758.0',
 '1140879760.0',
 '1140879762.0',
 '1140879818.0',
 '1141164276.0',
 '1140909368.0',
 '1140879824.0',
 '1140879794.0',
 '1140879798.0',
 '1140860750.0',
 '1140860696.0',
 '1140860882.0',
 '1140888552.0',
 '1140888560.0',
 '1140860806.0',
 '1140860904.0',
 '1141171336.0',
 '1141152998.0',
 '1140916356.0',
 '1141193282.0',
 '1141166006.0',
 '1141145660.0',
 '1140879806.0',
 '1140888510.0',
 '1140861088.0',
 '1140879810.0',
 '1140879802.0',
 '1140861276.0',
 '1141153026.0',
 '1140888686.0',
 '1140860532.0',
 '1140860470.0']

In [52]:
df_final = df[htn_drugcode_str].copy()
df_final

,eid,1140879822.0,1140860192.0,1140879842.0,1140866724.0,1140866738.0,1140879758.0,1140879760.0,1140879762.0,1140879818.0,...,1140879806.0,1140888510.0,1140861088.0,1140879810.0,1140879802.0,1140861276.0,1141153026.0,1140888686.0,1140860532.0,1140860470.0
0,6025371.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000027.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000039.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000040.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502481,6025327.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502482,6025339.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502483,6025340.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502484,6025353.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df_final.fillna(0, inplace = True)
df_final

,eid,1140879822.0,1140860192.0,1140879842.0,1140866724.0,1140866738.0,1140879758.0,1140879760.0,1140879762.0,1140879818.0,...,1140879806.0,1140888510.0,1140861088.0,1140879810.0,1140879802.0,1140861276.0,1141153026.0,1140888686.0,1140860532.0,1140860470.0
0,6025371.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000027.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000039.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000040.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502481,6025327.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502482,6025339.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502483,6025340.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502484,6025353.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#### test

d = {'A':[1, 0, 0, 0, 0], 
          'B':[0, 1, 0, 0, 1], 
          'C':[1, 1, 1, 0, 0], 
          'D':[0, 0, 0, 0, 1], 
          'E':[0, 0, 0, 0, 0], 
          'col':[1, 1, 1, 0, 1]}
testdf = pd.DataFrame(data = d, index = ['prsn1', 'prsn2', 'prsn3', 'prsn4', 'prsn5'])
testdf

In [ ]:
def create_htnmed(df):
    count = 0
    col = pd.DataFrame(df).columns.tolist()
    print(col)

    for c in col: 
        if df[str(c)] == 1: 
            count += 1
    if count > 0: 
        return 1
    else: 
        return 0

In [54]:
for val in htn_drugcode: 
    df_final.loc[df_final[str(val)] == 1, 'HTN_med'] = 1

In [55]:
df_final['HTN_med'].fillna(0, inplace = True)

In [56]:
df_final['HTN_med'].value_counts()

0.0    405344
1.0     97142
Name: HTN_med, dtype: int64

#### 2021.12.13 ARB + ACEi 분류 추가

In [34]:
# ARB, ACEi code list

arb_code = [1141171336.0, 1141152998.0, 1140916356.0, 1141193282.0, 1141166006.0, 1141145660.0]
acei_code = [1140860750.0, 1140860696.0, 1140860882.0, 1140888552.0, 1140888560.0, 1140860806.0, 1140860904.0]

In [57]:
# create other htn code

set_arb_code = set(arb_code)
set_acei_code = set(acei_code)
set_htn_drugcode = set(htn_drugcode)

set_other_htn_drugcode = set_htn_drugcode - set_acei_code - set_arb_code
other_htn_drugcode = list(set_other_htn_drugcode)
other_htn_drugcode

[1141153026.0,
 1140879810.0,
 1140879818.0,
 1140879758.0,
 1140879822.0,
 1140879760.0,
 1140879824.0,
 1140879762.0,
 1140888510.0,
 1140861276.0,
 1140860192.0,
 1140861088.0,
 1140879842.0,
 1140866724.0,
 1140888686.0,
 1140866738.0,
 1140879794.0,
 1141164276.0,
 1140860532.0,
 1140879798.0,
 1140909368.0,
 1140860470.0,
 1140879802.0,
 1140879806.0]

In [58]:
for val in arb_code: 
    df_final.loc[df_final[str(val)] == 1, 'ARB'] = 1

In [59]:
for val in acei_code: 
    df_final.loc[df_final[str(val)] == 1, 'ACEi'] = 1

In [60]:
for val in other_htn_drugcode: 
    df_final.loc[df_final[str(val)] == 1, 'Other_HTN_med'] = 1

In [61]:
df_final['ARB'].fillna(0, inplace = True)

In [62]:
df_final['ACEi'].fillna(0, inplace = True)

In [63]:
df_final['Other_HTN_med'].fillna(0, inplace = True)

In [64]:
df_final['ARB'].value_counts()

0.0    488934
1.0     13552
Name: ARB, dtype: int64

In [65]:
df_final['ACEi'].value_counts()

0.0    450828
1.0     51658
Name: ACEi, dtype: int64

In [66]:
df_final['Other_HTN_med'].value_counts()

0.0    444446
1.0     58040
Name: Other_HTN_med, dtype: int64

In [67]:
df_final['HTN_med'].value_counts()

0.0    405344
1.0     97142
Name: HTN_med, dtype: int64

In [68]:
df_final = df_final[['eid', 'ARB', 'ACEi', 'Other_HTN_med', 'HTN_med']]

In [69]:
df_final

,eid,ARB,ACEi,Other_HTN_med,HTN_med
0,6025371.0,0.0,0.0,0.0,0.0
1,1000015.0,1.0,0.0,0.0,1.0
2,1000027.0,0.0,0.0,0.0,0.0
3,1000039.0,0.0,1.0,0.0,1.0
4,1000040.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
502481,6025327.0,0.0,0.0,0.0,0.0
502482,6025339.0,0.0,0.0,0.0,0.0
502483,6025340.0,0.0,0.0,0.0,0.0
502484,6025353.0,0.0,1.0,0.0,1.0


In [70]:
df_final.to_csv('data\\ukb45783_drugpreprocessed_약학정보원기준.csv', encoding = 'euc-kr', index = False)

#### 2021.12.08 ~     
#### Last follow up date 생성

In [2]:
# 답 없음 eid는 num으로, date는 전부 character로 만들어서 sas로 가져가자

date_merged = pd.read_sas('SAS\\ukb_date_merged.sas7bdat', format = 'sas7bdat', encoding = 'cp1252')
date_merged

,23048-0.0,23048-1.0,30502-0.0,30502-1.0,30512-0.0,30512-1.0,30522-0.0,30522-1.0,30601-0.0,30601-1.0,...,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46,last_followup_date
0,NaN,NaN,2014-11-08,NaN,2014-11-08,NaN,2014-11-08,NaN,2016-01-10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2017-02-04,2016-10-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2014-12-01,NaN,2014-12-01,NaN,2014-12-01,NaN,2017-01-07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,2015-07-12,NaN,2015-07-12,NaN,2015-07-12,NaN,2017-03-07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,2015-01-31,NaN,2015-01-31,NaN,2015-01-31,NaN,2017-08-07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502470,NaN,NaN,2014-11-29,NaN,2014-11-29,NaN,2014-11-29,NaN,2016-05-10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502471,NaN,NaN,2015-02-07,NaN,2015-02-07,NaN,2015-02-07,NaN,2017-04-06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,NaN,NaN,2015-02-21,NaN,2015-02-21,NaN,2015-02-21,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,NaN,NaN,2015-09-17,NaN,2015-09-17,NaN,2015-09-17,NaN,2016-12-20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# null을 다 1900-01-01로 채우고, 날짜 -> 문자로 바꿔서 sas에서 가져온 데이터셋 

date_changed = pd.read_sas('SAS\\UKB_DATE_MERGED_DATETIME2.sas7bdat', format = 'sas7bdat', encoding = 'cp1252')
date_changed

,eid,23048-0.0,23048-1.0,30502-0.0,30502-1.0,30512-0.0,30512-1.0,30522-0.0,30522-1.0,30601-0.0,...,41281-0.37,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46
0,1000015.0,1900-01-01,1900-01-01,2014-11-08,1900-01-01,2014-11-08,1900-01-01,2014-11-08,1900-01-01,2016-01-10,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
1,1000027.0,1900-01-01,1900-01-01,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2017-02-04,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
2,1000039.0,1900-01-01,1900-01-01,2014-12-01,1900-01-01,2014-12-01,1900-01-01,2014-12-01,1900-01-01,2017-01-07,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
3,1000040.0,1900-01-01,1900-01-01,2015-07-12,1900-01-01,2015-07-12,1900-01-01,2015-07-12,1900-01-01,2017-03-07,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
4,1000053.0,1900-01-01,1900-01-01,2015-01-31,1900-01-01,2015-01-31,1900-01-01,2015-01-31,1900-01-01,2017-08-07,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502470,6025327.0,1900-01-01,1900-01-01,2014-11-29,1900-01-01,2014-11-29,1900-01-01,2014-11-29,1900-01-01,2016-05-10,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
502471,6025339.0,1900-01-01,1900-01-01,2015-02-07,1900-01-01,2015-02-07,1900-01-01,2015-02-07,1900-01-01,2017-04-06,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
502472,6025340.0,1900-01-01,1900-01-01,2015-02-21,1900-01-01,2015-02-21,1900-01-01,2015-02-21,1900-01-01,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
502473,6025353.0,1900-01-01,1900-01-01,2015-09-17,1900-01-01,2015-09-17,1900-01-01,2015-09-17,1900-01-01,2016-12-20,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01


In [4]:
df = date_changed.copy()

In [5]:
id_tmp = df[['eid']]

In [6]:
id_tmp

,eid
0,1000015.0
1,1000027.0
2,1000039.0
3,1000040.0
4,1000053.0
...,...
502470,6025327.0
502471,6025339.0
502472,6025340.0
502473,6025353.0


In [7]:
df.drop(['eid'], axis = 1, inplace = True)
df

,23048-0.0,23048-1.0,30502-0.0,30502-1.0,30512-0.0,30512-1.0,30522-0.0,30522-1.0,30601-0.0,30601-1.0,...,41281-0.37,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46
0,1900-01-01,1900-01-01,2014-11-08,1900-01-01,2014-11-08,1900-01-01,2014-11-08,1900-01-01,2016-01-10,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
1,1900-01-01,1900-01-01,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2017-02-04,2016-10-20,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
2,1900-01-01,1900-01-01,2014-12-01,1900-01-01,2014-12-01,1900-01-01,2014-12-01,1900-01-01,2017-01-07,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
3,1900-01-01,1900-01-01,2015-07-12,1900-01-01,2015-07-12,1900-01-01,2015-07-12,1900-01-01,2017-03-07,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
4,1900-01-01,1900-01-01,2015-01-31,1900-01-01,2015-01-31,1900-01-01,2015-01-31,1900-01-01,2017-08-07,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502470,1900-01-01,1900-01-01,2014-11-29,1900-01-01,2014-11-29,1900-01-01,2014-11-29,1900-01-01,2016-05-10,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
502471,1900-01-01,1900-01-01,2015-02-07,1900-01-01,2015-02-07,1900-01-01,2015-02-07,1900-01-01,2017-04-06,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
502472,1900-01-01,1900-01-01,2015-02-21,1900-01-01,2015-02-21,1900-01-01,2015-02-21,1900-01-01,1900-01-01,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01
502473,1900-01-01,1900-01-01,2015-09-17,1900-01-01,2015-09-17,1900-01-01,2015-09-17,1900-01-01,2016-12-20,1900-01-01,...,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01,1900-01-01


In [ ]:
df['30502-0.0'][0]

In [34]:
# Nan이 있으면 max value가 다 nan으로 잡힌다

df.fillna('0', inplace = True)

In [51]:
# datetime 형식으로 전부 바꾸기

df.iloc[:, 0:493] = df.iloc[:, 0:493].apply(pd.to_datetime, errors='coerce')

In [ ]:
# df.replace(0, '0', inplace = True)

In [52]:
df

,23048-0.0,23048-1.0,30502-0.0,30502-1.0,30512-0.0,30512-1.0,30522-0.0,30522-1.0,30601-0.0,30601-1.0,30611-0.0,30611-1.0,30621-0.0,30621-1.0,30631-0.0,30631-1.0,30651-0.0,30651-1.0,30661-0.0,30661-1.0,30671-0.0,30671-1.0,30681-0.0,30681-1.0,30691-0.0,30691-1.0,30701-0.0,30701-1.0,30711-0.0,30711-1.0,30721-0.0,30721-1.0,30731-0.0,30731-1.0,30741-0.0,30741-1.0,30751-0.0,30751-1.0,30761-0.0,30761-1.0,30771-0.0,30771-1.0,30781-0.0,30781-1.0,30791-0.0,30791-1.0,30801-0.0,30801-1.0,30811-0.0,30811-1.0,30821-0.0,30821-1.0,30831-0.0,30831-1.0,30841-0.0,30841-1.0,30851-0.0,30851-1.0,30861-0.0,30861-1.0,30871-0.0,30871-1.0,30881-0.0,30881-1.0,30891-0.0,30891-1.0,53-0.0,53-1.0,53-2.0,53-3.0,20078-0.0,20078-1.0,20078-2.0,20078-3.0,20078-4.0,20400-0.0,22500-0.0,22700-0.0,22700-0.1,22700-0.2,22700-0.3,22700-0.4,22700-0.5,22700-0.6,22700-0.7,22700-0.8,22700-0.9,22700-0.10,22700-0.11,22700-0.12,22700-0.13,22700-0.14,30532-0.0,30532-1.0,40000-0.0,40000-1.0,40005-0.0,40005-1.0,40005-2.0,40005-3.0,40005-4.0,40005-5.0,40005-6.0,40005-7.0,40005-8.0,40005-9.0,40005-10.0,40005-11.0,40005-12.0,40005-13.0,40005-14.0,40005-15.0,40005-16.0,42000-0.0,42002-0.0,42004-0.0,42006-0.0,42008-0.0,42010-0.0,42012-0.0,41262-0.0,41262-0.1,41262-0.2,41262-0.3,41262-0.4,41262-0.5,41262-0.6,41262-0.7,41262-0.8,41262-0.9,41262-0.10,41262-0.11,41262-0.12,41262-0.13,41262-0.14,41262-0.15,41262-0.16,41262-0.17,41262-0.18,41262-0.19,41262-0.20,41262-0.21,41262-0.22,41262-0.23,41262-0.24,41262-0.25,41262-0.26,41262-0.27,41262-0.28,41262-0.29,41262-0.30,41262-0.31,41262-0.32,41262-0.33,41262-0.34,41262-0.35,41262-0.36,41262-0.37,41262-0.38,41262-0.39,41262-0.40,41262-0.41,41262-0.42,41262-0.43,41262-0.44,41262-0.45,41262-0.46,41262-0.47,41262-0.48,41262-0.49,41262-0.50,41262-0.51,41262-0.52,41262-0.53,41262-0.54,41262-0.55,41262-0.56,41262-0.57,41262-0.58,41262-0.59,41262-0.60,41262-0.61,41262-0.62,41262-0.63,41262-0.64,41262-0.65,41262-0.66,41262-0.67,41262-0.68,41262-0.69,41262-0.70,41262-0.71,41262-0.72,41262-0.73,41262-0.74,41263-0.0,41263-0.1,41263-0.2,41263-0.3,41263-0.4,41263-0.5,41263-0.6,41263-0.7,41263-0.8,41263-0.9,41263-0.10,41263-0.11,41263-0.12,41263-0.13,41263-0.14,41263-0.15,41263-0.16,41263-0.17,41263-0.18,41263-0.19,41263-0.20,41263-0.21,41263-0.22,41263-0.23,41263-0.24,41263-0.25,41263-0.26,41263-0.27,41280-0.0,41280-0.1,41280-0.2,41280-0.3,41280-0.4,41280-0.5,41280-0.6,41280-0.7,41280-0.8,41280-0.9,41280-0.10,41280-0.11,41280-0.12,41280-0.13,41280-0.14,41280-0.15,41280-0.16,41280-0.17,41280-0.18,41280-0.19,41280-0.20,41280-0.21,41280-0.22,41280-0.23,41280-0.24,41280-0.25,41280-0.26,41280-0.27,41280-0.28,41280-0.29,41280-0.30,41280-0.31,41280-0.32,41280-0.33,41280-0.34,41280-0.35,41280-0.36,41280-0.37,41280-0.38,41280-0.39,41280-0.40,41280-0.41,41280-0.42,41280-0.43,41280-0.44,41280-0.45,41280-0.46,41280-0.47,41280-0.48,41280-0.49,41280-0.50,41280-0.51,41280-0.52,41280-0.53,41280-0.54,41280-0.55,41280-0.56,41280-0.57,41280-0.58,41280-0.59,41280-0.60,41280-0.61,41280-0.62,41280-0.63,41280-0.64,41280-0.65,41280-0.66,41280-0.67,41280-0.68,41280-0.69,41280-0.70,41280-0.71,41280-0.72,41280-0.73,41280-0.74,41280-0.75,41280-0.76,41280-0.77,41280-0.78,41280-0.79,41280-0.80,41280-0.81,41280-0.82,41280-0.83,41280-0.84,41280-0.85,41280-0.86,41280-0.87,41280-0.88,41280-0.89,41280-0.90,41280-0.91,41280-0.92,41280-0.93,41280-0.94,41280-0.95,41280-0.96,41280-0.97,41280-0.98,41280-0.99,41280-0.100,41280-0.101,41280-0.102,41280-0.103,41280-0.104,41280-0.105,41280-0.106,41280-0.107,41280-0.108,41280-0.109,41280-0.110,41280-0.111,41280-0.112,41280-0.113,41280-0.114,41280-0.115,41280-0.116,41280-0.117,41280-0.118,41280-0.119,41280-0.120,41280-0.121,41280-0.122,41280-0.123,41280-0.124,41280-0.125,41280-0.126,41280-0.127,41280-0.128,41280-0.129,41280-0.130,41280-0.131,41280-0.132,41280-0.133,41280-0.134,41280-0.135,41280-0.136,41280-0.137,41280-0.138,41280-0.139,41280-0.140,41280-0.141,41280-0.142,41280-0.143,41280-0.144,41280-0.145,41280-0.146,41280-0.147,41280-0.148,41280-0.

In [53]:
df = pd.concat([id_tmp, df], axis = 1)
df

,eid,23048-0.0,23048-1.0,30502-0.0,30502-1.0,30512-0.0,30512-1.0,30522-0.0,30522-1.0,30601-0.0,30601-1.0,30611-0.0,30611-1.0,30621-0.0,30621-1.0,30631-0.0,30631-1.0,30651-0.0,30651-1.0,30661-0.0,30661-1.0,30671-0.0,30671-1.0,30681-0.0,30681-1.0,30691-0.0,30691-1.0,30701-0.0,30701-1.0,30711-0.0,30711-1.0,30721-0.0,30721-1.0,30731-0.0,30731-1.0,30741-0.0,30741-1.0,30751-0.0,30751-1.0,30761-0.0,30761-1.0,30771-0.0,30771-1.0,30781-0.0,30781-1.0,30791-0.0,30791-1.0,30801-0.0,30801-1.0,30811-0.0,30811-1.0,30821-0.0,30821-1.0,30831-0.0,30831-1.0,30841-0.0,30841-1.0,30851-0.0,30851-1.0,30861-0.0,30861-1.0,30871-0.0,30871-1.0,30881-0.0,30881-1.0,30891-0.0,30891-1.0,53-0.0,53-1.0,53-2.0,53-3.0,20078-0.0,20078-1.0,20078-2.0,20078-3.0,20078-4.0,20400-0.0,22500-0.0,22700-0.0,22700-0.1,22700-0.2,22700-0.3,22700-0.4,22700-0.5,22700-0.6,22700-0.7,22700-0.8,22700-0.9,22700-0.10,22700-0.11,22700-0.12,22700-0.13,22700-0.14,30532-0.0,30532-1.0,40000-0.0,40000-1.0,40005-0.0,40005-1.0,40005-2.0,40005-3.0,40005-4.0,40005-5.0,40005-6.0,40005-7.0,40005-8.0,40005-9.0,40005-10.0,40005-11.0,40005-12.0,40005-13.0,40005-14.0,40005-15.0,40005-16.0,42000-0.0,42002-0.0,42004-0.0,42006-0.0,42008-0.0,42010-0.0,42012-0.0,41262-0.0,41262-0.1,41262-0.2,41262-0.3,41262-0.4,41262-0.5,41262-0.6,41262-0.7,41262-0.8,41262-0.9,41262-0.10,41262-0.11,41262-0.12,41262-0.13,41262-0.14,41262-0.15,41262-0.16,41262-0.17,41262-0.18,41262-0.19,41262-0.20,41262-0.21,41262-0.22,41262-0.23,41262-0.24,41262-0.25,41262-0.26,41262-0.27,41262-0.28,41262-0.29,41262-0.30,41262-0.31,41262-0.32,41262-0.33,41262-0.34,41262-0.35,41262-0.36,41262-0.37,41262-0.38,41262-0.39,41262-0.40,41262-0.41,41262-0.42,41262-0.43,41262-0.44,41262-0.45,41262-0.46,41262-0.47,41262-0.48,41262-0.49,41262-0.50,41262-0.51,41262-0.52,41262-0.53,41262-0.54,41262-0.55,41262-0.56,41262-0.57,41262-0.58,41262-0.59,41262-0.60,41262-0.61,41262-0.62,41262-0.63,41262-0.64,41262-0.65,41262-0.66,41262-0.67,41262-0.68,41262-0.69,41262-0.70,41262-0.71,41262-0.72,41262-0.73,41262-0.74,41263-0.0,41263-0.1,41263-0.2,41263-0.3,41263-0.4,41263-0.5,41263-0.6,41263-0.7,41263-0.8,41263-0.9,41263-0.10,41263-0.11,41263-0.12,41263-0.13,41263-0.14,41263-0.15,41263-0.16,41263-0.17,41263-0.18,41263-0.19,41263-0.20,41263-0.21,41263-0.22,41263-0.23,41263-0.24,41263-0.25,41263-0.26,41263-0.27,41280-0.0,41280-0.1,41280-0.2,41280-0.3,41280-0.4,41280-0.5,41280-0.6,41280-0.7,41280-0.8,41280-0.9,41280-0.10,41280-0.11,41280-0.12,41280-0.13,41280-0.14,41280-0.15,41280-0.16,41280-0.17,41280-0.18,41280-0.19,41280-0.20,41280-0.21,41280-0.22,41280-0.23,41280-0.24,41280-0.25,41280-0.26,41280-0.27,41280-0.28,41280-0.29,41280-0.30,41280-0.31,41280-0.32,41280-0.33,41280-0.34,41280-0.35,41280-0.36,41280-0.37,41280-0.38,41280-0.39,41280-0.40,41280-0.41,41280-0.42,41280-0.43,41280-0.44,41280-0.45,41280-0.46,41280-0.47,41280-0.48,41280-0.49,41280-0.50,41280-0.51,41280-0.52,41280-0.53,41280-0.54,41280-0.55,41280-0.56,41280-0.57,41280-0.58,41280-0.59,41280-0.60,41280-0.61,41280-0.62,41280-0.63,41280-0.64,41280-0.65,41280-0.66,41280-0.67,41280-0.68,41280-0.69,41280-0.70,41280-0.71,41280-0.72,41280-0.73,41280-0.74,41280-0.75,41280-0.76,41280-0.77,41280-0.78,41280-0.79,41280-0.80,41280-0.81,41280-0.82,41280-0.83,41280-0.84,41280-0.85,41280-0.86,41280-0.87,41280-0.88,41280-0.89,41280-0.90,41280-0.91,41280-0.92,41280-0.93,41280-0.94,41280-0.95,41280-0.96,41280-0.97,41280-0.98,41280-0.99,41280-0.100,41280-0.101,41280-0.102,41280-0.103,41280-0.104,41280-0.105,41280-0.106,41280-0.107,41280-0.108,41280-0.109,41280-0.110,41280-0.111,41280-0.112,41280-0.113,41280-0.114,41280-0.115,41280-0.116,41280-0.117,41280-0.118,41280-0.119,41280-0.120,41280-0.121,41280-0.122,41280-0.123,41280-0.124,41280-0.125,41280-0.126,41280-0.127,41280-0.128,41280-0.129,41280-0.130,41280-0.131,41280-0.132,41280-0.133,41280-0.134,41280-0.135,41280-0.136,41280-0.137,41280-0.138,41280-0.139,41280-0.140,41280-0.141,41280-0.142,41280-0.143,41280-0.144,41280-0.145,41280-0.146,41280-0.147,41280-0.148,4128

In [54]:
# sas로 가져가기

df.to_csv('data\\ukb_date_merged_DATETIME.csv', encoding = 'euc-kr', index = False)

In [35]:
# # datetime 형식인 열들 str로 바꾸기 - 안됨

# df.iloc[:, 67:120] = df.iloc[:, 67:120].strftime("%Y-%m-%d")

AttributeError: 'DataFrame' object has no attribute 'strftime'

In [36]:
# # 왜 안바뀌지 ..? 전체 다 해보자  - DATETIME 형식을 astype로 str로 바꿀 순 없는 듯 하다

# all_columns = df.columns.tolist()
# df[all_columns] = df[all_columns].astype(str)

In [40]:
df.replace({pd.NaT: np.NaN}, inplace = True)

In [41]:
df.iloc[:, 67:120]

,53-1.0,53-2.0,53-3.0,20078-0.0,20078-1.0,20078-2.0,20078-3.0,20078-4.0,20400-0.0,22500-0.0,...,40005-14.0,40005-15.0,40005-16.0,42000-0.0,42002-0.0,42004-0.0,42006-0.0,42008-0.0,42010-0.0,42012-0.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-20 00:00:00,NaN,2015-08-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-09-20 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,2011-03-10 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502471,NaN,NaN,NaN,2009-09-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-09 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# 왜 안돼 ....

last_followup_date = np.nanmax(df, axis = 1)

In [13]:
last_followup_date = pd.DataFrame(last_followup_date)

In [14]:
last_followup_date

,0
0,2020-09-18
1,2017-02-05
2,2017-10-19
3,2017-03-08
4,2020-02-13
...,...
502470,2016-05-10
502471,2017-04-07
502472,2015-02-21
502473,2019-07-27


In [16]:
df_final = pd.concat([id_tmp, last_followup_date], axis = 1)
df_final

,eid,0
0,1000015.0,2020-09-18
1,1000027.0,2017-02-05
2,1000039.0,2017-10-19
3,1000040.0,2017-03-08
4,1000053.0,2020-02-13
...,...,...
502470,6025327.0,2016-05-10
502471,6025339.0,2017-04-07
502472,6025340.0,2015-02-21
502473,6025353.0,2019-07-27


In [17]:
df_final.rename(columns = {0 : 'LastFollowUpDate'}, inplace = True)
df_final

,eid,LastFollowUpDate
0,1000015.0,2020-09-18
1,1000027.0,2017-02-05
2,1000039.0,2017-10-19
3,1000040.0,2017-03-08
4,1000053.0,2020-02-13
...,...,...
502470,6025327.0,2016-05-10
502471,6025339.0,2017-04-07
502472,6025340.0,2015-02-21
502473,6025353.0,2019-07-27


In [20]:
len(df_final.LastFollowUpDate.unique())

# 50만명인데 3031..? 이게 맞나? 

3031

In [21]:
# LastFollowUpDate created

df_final.to_csv('data\\ukb_date_LastFollowUpDatepreprocessed.csv', encoding = 'euc-kr', index = False)

In [ ]:
pd.set_option('display.max_column', None)
k = df.iloc[[0]]
k

In [ ]:
np.nanmax(k, axis = 1)

In [ ]:
ary = ['2012-05-31', '2013-03-03', '2016-05-05', np.nan]
ary

In [ ]:
ary

In [ ]:
df['30502-0.0']

#### 2021.12.09 ~
#### ukb46987_new_icd9 에서 icd9 코드로 ckd 진단받은 사람 찾아서 자르기

In [118]:
ukb46987_icd9 = pd.read_sas('SAS\\ukb46987_new_icd9.sas7bdat', format = 'sas7bdat', encoding = 'cp1252')
ukb46987_icd9

,eid,41271-0.0,41271-0.1,41271-0.2,41271-0.3,41271-0.4,41271-0.5,41271-0.6,41271-0.7,41271-0.8,...,41281-0.37,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46
0,1000015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,6180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
df = ukb46987_icd9.copy()

In [5]:
# pd.set_option('display.max_column', None)
# df

,eid,41271-0.0,41271-0.1,41271-0.2,41271-0.3,41271-0.4,41271-0.5,41271-0.6,41271-0.7,41271-0.8,41271-0.9,41271-0.10,41271-0.11,41271-0.12,41271-0.13,41271-0.14,41271-0.15,41271-0.16,41271-0.17,41271-0.18,41271-0.19,41271-0.20,41271-0.21,41271-0.22,41271-0.23,41271-0.24,41271-0.25,41271-0.26,41271-0.27,41271-0.28,41271-0.29,41271-0.30,41271-0.31,41271-0.32,41271-0.33,41271-0.34,41271-0.35,41271-0.36,41271-0.37,41271-0.38,41271-0.39,41271-0.40,41271-0.41,41271-0.42,41271-0.43,41271-0.44,41271-0.45,41271-0.46,41281-0.0,41281-0.1,41281-0.2,41281-0.3,41281-0.4,41281-0.5,41281-0.6,41281-0.7,41281-0.8,41281-0.9,41281-0.10,41281-0.11,41281-0.12,41281-0.13,41281-0.14,41281-0.15,41281-0.16,41281-0.17,41281-0.18,41281-0.19,41281-0.20,41281-0.21,41281-0.22,41281-0.23,41281-0.24,41281-0.25,41281-0.26,41281-0.27,41281-0.28,41281-0.29,41281-0.30,41281-0.31,41281-0.32,41281-0.33,41281-0.34,41281-0.35,41281-0.36,41281-0.37,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46,75310,75311,75312,5859
0,1000015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,1000027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,1000053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
502472,6025340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [121]:
# df2 : icd9 진단여부
df2 = df.iloc[:, :48]
df2

,eid,41271-0.0,41271-0.1,41271-0.2,41271-0.3,41271-0.4,41271-0.5,41271-0.6,41271-0.7,41271-0.8,...,41271-0.37,41271-0.38,41271-0.39,41271-0.40,41271-0.41,41271-0.42,41271-0.43,41271-0.44,41271-0.45,41271-0.46
0,1000015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,6180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
# df3 : icd9 진단날짜

df3 = df.iloc[:, 48:]
df3

,41281-0.0,41281-0.1,41281-0.2,41281-0.3,41281-0.4,41281-0.5,41281-0.6,41281-0.7,41281-0.8,41281-0.9,...,41281-0.37,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,1989-04-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### icd9 glaucoma 진단코드 

365 Glaucoma

- 365.0 Borderline glaucoma (glaucoma suspect)    
- 365.1 Open-angle glaucoma 
- 365.2 Primary angle-closure glaucoma 
- 365.3 Corticosteroid-induced glaucoma
- 365.4 Glaucoma associated with congenital anomalies dystrophies and systemic syndromes
- 365.5 Glaucoma associated with disorders of the lens    
- 365.6 Glaucoma associated with other ocular disorders    
- 365.7 Glaucoma stage   
- 365.8 Other specified forms of glaucoma    
- 365.9 Unspecified glaucoma    

In [123]:
df['3650'] = 0
df['3651'] = 0
df['3652'] = 0
df['3653'] = 0
df['3654'] = 0
df['3655'] = 0
df['3656'] = 0
df['3657'] = 0
df['3658'] = 0
df['3659'] = 0

In [124]:
colnames = df2.columns.tolist()
colnames.remove('eid')

for col in colnames: 
    df2.loc[df2[col] == '3650', '3650'] = 1  # 전체 222개 칼럼 순회하면서 3650 있으면 1 넣어주기
    df2.loc[df2[col] == '3650', '3650_index'] = col[6:]  # 41281 변수에서 3650 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [126]:
for col in colnames: 
    df2.loc[df2[col] == '3651', '3651'] = 1  # 전체 222개 칼럼 순회하면서 3651 있으면 1 넣어주기
    df2.loc[df2[col] == '3651', '3651_index'] = col[6:]  # 41281 변수에서 3651 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [127]:
for col in colnames: 
    df2.loc[df2[col] == '3652', '3652'] = 1  # 전체 222개 칼럼 순회하면서 3652 있으면 1 넣어주기
    df2.loc[df2[col] == '3652', '3652_index'] = col[6:]  # 41281 변수에서 3652 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [128]:
for col in colnames: 
    df2.loc[df2[col] == '3653', '3653'] = 1  # 전체 222개 칼럼 순회하면서 3653 있으면 1 넣어주기
    df2.loc[df2[col] == '3653', '3653_index'] = col[6:]  # 41281 변수에서 3653 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [129]:
for col in colnames: 
    df2.loc[df2[col] == '3654', '3654'] = 1  # 전체 222개 칼럼 순회하면서 3654 있으면 1 넣어주기
    df2.loc[df2[col] == '3654', '3654_index'] = col[6:]  # 41281 변수에서 3654 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [130]:
for col in colnames: 
    df2.loc[df2[col] == '3655', '3655'] = 1  # 전체 222개 칼럼 순회하면서 3655 있으면 1 넣어주기
    df2.loc[df2[col] == '3655', '3655_index'] = col[6:]  # 41281 변수에서 3655 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [131]:
for col in colnames: 
    df2.loc[df2[col] == '3656', '3656'] = 1  # 전체 222개 칼럼 순회하면서 3656 있으면 1 넣어주기
    df2.loc[df2[col] == '3656', '3656_index'] = col[6:]  # 41281 변수에서 3656 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [132]:
for col in colnames: 
    df2.loc[df2[col] == '3657', '3657'] = 1  # 전체 222개 칼럼 순회하면서 3657 있으면 1 넣어주기
    df2.loc[df2[col] == '3657', '3657_index'] = col[6:]  # 41281 변수에서 3657 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [133]:
for col in colnames: 
    df2.loc[df2[col] == '3658', '3658'] = 1  # 전체 222개 칼럼 순회하면서 3658 있으면 1 넣어주기
    df2.loc[df2[col] == '3658', '3658_index'] = col[6:]  # 41281 변수에서 3658 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [134]:
for col in colnames: 
    df2.loc[df2[col] == '3659', '3659'] = 1  # 전체 222개 칼럼 순회하면서 3659 있으면 1 넣어주기
    df2.loc[df2[col] == '3659', '3659_index'] = col[6:]  # 41281 변수에서 3659 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [135]:
print(df2["3650"].value_counts())
print(df2["3650_index"].isna().sum())

1.0    4
Name: 3650, dtype: int64
502472


In [136]:
print(df2["3651"].value_counts())
print(df2["3651_index"].isna().sum())

1.0    8
Name: 3651, dtype: int64
502468


In [137]:
print(df2["3652"].value_counts())
print(df2["3652_index"].isna().sum())

1.0    8
Name: 3652, dtype: int64
502468


In [138]:
print(df2["3653"].value_counts())
print(df2["3653_index"].isna().sum())

Series([], Name: 3653, dtype: int64)
502476


In [139]:
print(df2["3654"].value_counts())
print(df2["3654_index"].isna().sum())

Series([], Name: 3654, dtype: int64)
502476


In [140]:
print(df2["3655"].value_counts())
print(df2["3655_index"].isna().sum())

1.0    1
Name: 3655, dtype: int64
502475


In [141]:
print(df2["3656"].value_counts())
print(df2["3656_index"].isna().sum())

1.0    3
Name: 3656, dtype: int64
502473


In [142]:
print(df2["3657"].value_counts())
print(df2["3657_index"].isna().sum())

Series([], Name: 3657, dtype: int64)
502476


In [143]:
print(df2["3658"].value_counts())
print(df2["3658_index"].isna().sum())

Series([], Name: 3658, dtype: int64)
502476


In [144]:
print(df2["3659"].value_counts())
print(df2["3659_index"].isna().sum())

1.0    24
Name: 3659, dtype: int64
502452


In [25]:
# glaucoma 진단자 총 48명

In [159]:
# 48명 id 리스트로 만들기 -> 중복자 제거하니 41명

icd9_ckd_diagnosed = df2.loc[((df2['3650'] == 1) | (df2['3651'] == 1) | (df2['3652'] == 1) | (df2['3653'] == 1) | (df2['3654'] == 1) | (df2['3655'] == 1)
| (df2['3656'] == 1) | (df2['3657'] == 1) | (df2['3658'] == 1) | (df2['3659'] == 1)), 'eid'].tolist()
icd9_ckd_diagnosed = list(dict.fromkeys(icd9_ckd_diagnosed))
icd9_ckd_diagnosed

['1145519',
 '1147093',
 '1292905',
 '1614844',
 '1753733',
 '1827041',
 '1971539',
 '2332508',
 '2383831',
 '2408819',
 '2744147',
 '2855489',
 '2947124',
 '3103867',
 '3388611',
 '3439966',
 '3669679',
 '3758036',
 '3945168',
 '3986583',
 '4039752',
 '4044781',
 '4059037',
 '4123336',
 '4171851',
 '4383643',
 '4473000',
 '4494103',
 '4554560',
 '4601616',
 '4608219',
 '5023948',
 '5228422',
 '5279912',
 '5302701',
 '5302836',
 '5368953',
 '5423356',
 '5701595',
 '5729779',
 '5954784']

In [157]:
icd10 = pd.read_csv('0. data\\ukb46987_H40preprocessed.csv')
icd10

,eid,H40_Date,H40
0,1000015,0,0
1,1000027,0,0
2,1000039,0,0
3,1000040,0,0
4,1000053,0,0
...,...,...,...
502471,6025339,0,0
502472,6025340,0,0
502473,6025353,0,0
502474,6025364,0,0


In [158]:
# icd9 진단받은 41명 최종 데이터프레임에서 자르기

final = icd10.loc[~icd10['eid'].isin(icd9_ckd_diagnosed)].reset_index(drop = True)
final
# 원래 없다

,eid,H40_Date,H40
0,1000015,0,0
1,1000027,0,0
2,1000039,0,0
3,1000040,0,0
4,1000053,0,0
...,...,...,...
502471,6025339,0,0
502472,6025340,0,0
502473,6025353,0,0
502474,6025364,0,0


In [160]:
# recheck - 원래 없다

icd10.loc[icd10['eid'] == '1145519']

,eid,H40_Date,H40
